<a href="https://colab.research.google.com/github/freezer2019/Analytics/blob/main/Stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install geemap

In [ ]:
import ee
import geemap
import pandas as pd
import numpy as np
import math
import datetime as dt

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=oSt6Owmzy4E4vKyobqCHUMV3kURXdkSh-Qj-rBq4C7A&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6I89bML_hk6Wu0q-fb7iNDrQjynhrAo6CW-JIgmxKme8FVsh0CAMs

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
Geometry1 =ee.Geometry.Polygon([[[-7.763444455294805,32.14083021642846], [-7.7633371669342095,32.14038507155666],
                                 [-7.762596877246099,32.140485002227244], [-7.762704165606695,32.140939231167884], 
                                 [-7.763444455294805,32.14083021642846]]])

Geometry2 =ee.Geometry.Polygon([[[-7.763310344844061,32.14030103896987], [-7.763203056483465,32.13990131482433], 
                                 [-7.762484224467474,32.14001033067415], [-7.76258614841004,32.1404145966463],
                                 [-7.763310344844061,32.14030103896987]]])

Geometry3 =ee.Geometry.Polygon([[[-7.762553961901861,32.14036917359273], [-7.762473495631414,32.14001941532244], 
                                 [-7.761674197344976,32.14018293883656], [-7.761797578959661,32.1405554079686], 
                                 [-7.762553961901861,32.14036917359273]]])


In [ ]:
satdataset = (ee.ImageCollection("COPERNICUS/S2_SR")
              .filterBounds (Geometry1)
              .filter(ee.Filter.date('2020-04-01', '2021-03-01'))
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
              .map(lambda imag: imag.clip(Geometry1)))

In [ ]:
satdataset.size().getInfo()

49

In [ ]:
satdataset.first().getInfo()

{'bands': [{'crs': 'EPSG:32629',
   'crs_transform': [60, 0, 600000, 0, -60, 3600000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [2, 2],
   'id': 'B1',
   'origin': [277, 721]},
  {'crs': 'EPSG:32629',
   'crs_transform': [10, 0, 600000, 0, -10, 3600000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [9, 7],
   'id': 'B2',
   'origin': [1662, 4327]},
  {'crs': 'EPSG:32629',
   'crs_transform': [10, 0, 600000, 0, -10, 3600000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [9, 7],
   'id': 'B3',
   'origin': [1662, 4327]},
  {'crs': 'EPSG:32629',
   'crs_transform': [10, 0, 600000, 0, -10, 3600000],
   'data_type': {'max': 65535,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [9, 7],
   'id': 'B4',
   'origin': [1662, 4327]},
  {'crs': 'EPSG:32629',
   'c

In [ ]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

In [ ]:
def addNDWI(image):
  ndwi = image.normalizedDifference(['B8', 'B3']).rename('NDWI')
  return image.addBands(ndwi)

In [ ]:
def addSAVI(image):
  savi = (((image.select("B8").subtract(image.select("B4"))).divide(image.select("B8").add(image.select("B4")).add(0.428))).multiply(1.428)).rename('SAVI')
  return image.addBands(savi)

In [ ]:
Dataset1 = satdataset.map(addNDVI).map(addNDWI).map(addSAVI)

In [ ]:
NDVI_data_Liste= Dataset1.toList(Dataset1.size())
print(NDVI_data_Liste.size().getInfo()),
NDVI_data_Liste

49


In [ ]:
NDWI_data_Liste= Dataset1.toList(Dataset1.size())
print(NDWI_data_Liste.size().getInfo()),
NDWI_data_Liste

49


In [ ]:
SAVI_data_Liste= Dataset1.toList(Dataset1.size())
print(NDWI_data_Liste.size().getInfo()),
NDWI_data_Liste

49


In [ ]:
NDVIListe = []
NDWIListe = []
SAVIListe = []
Date = []
Hourtime= []

In [ ]:
for i in range(NDVI_data_Liste.size().getInfo()):
  image=ee.Image(NDVI_data_Liste.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometry1,
      maxPixels=1e15,
      scale=30)
  NDVIListe.append(np.mean(latitudeLogititude.getInfo().get('NDVI')))
  Date.append(image.getInfo()["properties"]['DATATAKE_IDENTIFIER'])

In [ ]:
for i in range(NDWI_data_Liste.size().getInfo()):
  image=ee.Image(NDWI_data_Liste.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometry1,
      maxPixels=1e15,
      scale=30)
  NDWIListe.append(np.mean(latitudeLogititude.getInfo().get('NDWI')))

In [ ]:
for i in range(SAVI_data_Liste.size().getInfo()):
  image=ee.Image(SAVI_data_Liste.get(i))
  latlon_image = ee.Image.pixelLonLat().addBands(image)
  latitudeLogititude = latlon_image.reduceRegion(
      reducer=ee.Reducer.toList(),
      geometry=Geometry1,
      maxPixels=1e15,
      scale=30)
  SAVIListe.append(np.mean(latitudeLogititude.getInfo().get('SAVI')))

In [ ]:
column_names = ['Hourtime', 'Date', 'Air_temperature', 'Soil_temperature', 'Precipitation', 'NDVI', 'NDWI', 'SAVI', 'Leaf_Area_index', 'Land_surface_temperature', 'Volumetric_Soil_Moisture']

In [ ]:
dataset = pd.DataFrame(columns = column_names)

In [ ]:
Date1 = [x.replace('GS2A_', '') for x in Date]
Date1 = [x.replace('GS2B_', '') for x in Date1]
Date1 = [x.replace('_', '') for x in Date1]
Date1 = [x.replace('T', '') for x in Date1]
Date1 = [ x[0:14] for x in Date1 ]
Hourtime = [x[8:14] for x in Date1]
Date1 = [ x[0:8] for x in Date1 ]

In [ ]:
dataset["NDVI"] = NDVIListe
dataset["NDWI"] = NDWIListe
dataset["SAVI"] = SAVIListe

In [ ]:
result1 = [2.88 * x + 1.14 * (1 + 0.104 * math.exp(4.1 * x)) for x in NDWIListe]
dataset["Leaf_Area_index"] = result1

In [ ]:
Hourtime1 = [ x[:2] + ':' + x[2:] for x in Hourtime ]
Hourtime1 = [ x[:5] + ':' + x[5:] for x in Hourtime1 ] 

In [ ]:
dataset["Hourtime"] = Hourtime1
dataset["Date"] = pd.to_datetime(Date1, format='%Y-%m-%d', errors='ignore')

In [ ]:
dataset.head()

,Hourtime,Date,Air_temperature,Soil_temperature,Precipitation,NDVI,NDWI,SAVI,Leaf_Area_index,Land_surface_temperature,Volumetric_Soil_Moisture
0,11:06:21,2020-04-06,NaN,NaN,NaN,0.320214,0.436073,0.457228,3.104510,NaN,NaN
1,11:06:19,2020-05-01,NaN,NaN,NaN,0.321962,0.435299,0.459722,3.100036,NaN,NaN
2,11:06:21,2020-05-06,NaN,NaN,NaN,0.337963,0.442432,0.482572,3.141542,NaN,NaN
3,11:06:19,2020-05-11,NaN,NaN,NaN,0.222053,0.385900,0.317063,2.828257,NaN,NaN
4,11:06:19,2020-05-21,NaN,NaN,NaN,0.189151,0.368886,0.270085,2.740386,NaN,NaN


In [ ]:
weatherdata = pd.read_excel('weatherdata.xlsx', index_col=0)  

In [ ]:
weatherdata.head()

,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions
Name,,,,,,,,,,,,,,,,
"بن جرير, مراكش - آسفي, المغرب",04/01/2020 00:00:00,59.0,59.0,59.0,NaN,NaN,0.0,NaN,NaN,8.1,270.0,NaN,5.0,89.0,72.03,Overcast
"بن جرير, مراكش - آسفي, المغرب",04/01/2020 01:00:00,57.0,57.0,57.0,NaN,NaN,0.0,NaN,NaN,9.9,275.0,NaN,5.0,74.3,81.33,Partially cloudy
"بن جرير, مراكش - آسفي, المغرب",04/01/2020 02:00:00,55.4,55.4,55.4,NaN,NaN,0.0,NaN,NaN,3.4,250.0,NaN,5.0,50.0,82.01,Partially cloudy
"بن جرير, مراكش - آسفي, المغرب",04/01/2020 03:00:00,55.4,55.4,55.4,NaN,NaN,0.0,NaN,NaN,1.1,NaN,NaN,5.0,50.0,82.01,Partially cloudy
"بن جرير, مراكش - آسفي, المغرب",04/01/2020 04:00:00,55.2,55.2,55.2,NaN,NaN,0.0,NaN,NaN,3.5,300.0,NaN,5.0,50.0,82.80,Partially cloudy


In [ ]:
Date2 = []
Hourtime2 = []
Date2 = [x[:10] for x in weatherdata["Date time"]]
Hourtime2 = [x[11:] for x in weatherdata["Date time"]]
Date2 = [x.replace('/', '-') for x in Date2]
Date2 = [ x[6:] + '-' + x[:2] + '-' + x[3:5] for x in Date2 ]
weatherdata["Date"] = pd.to_datetime(Date2, format='%Y-%m-%d', errors='ignore')
weatherdata['Hour'] = Hourtime2

In [ ]:
weatherdata = weatherdata.drop(columns='Date time', axis = 1)
weatherdata.head(2)

,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions,Date,Hour
Name,,,,,,,,,,,,,,,,,
"بن جرير, مراكش - آسفي, المغرب",59.0,59.0,59.0,NaN,NaN,0.0,NaN,NaN,8.1,270.0,NaN,5.0,89.0,72.03,Overcast,2020-04-01,00:00:00
"بن جرير, مراكش - آسفي, المغرب",57.0,57.0,57.0,NaN,NaN,0.0,NaN,NaN,9.9,275.0,NaN,5.0,74.3,81.33,Partially cloudy,2020-04-01,01:00:00


In [ ]:
new_data = pd.merge(dataset,weatherdata[weatherdata["Hour"]=="11:00:00"][["Temperature", "Precipitation", "Date"]], on='Date',suffixes=('_dataset', ''))

In [ ]:
new_data.head(10)

,Hourtime,Date,Air_temperature,Soil_temperature,Precipitation_dataset,NDVI,NDWI,SAVI,Leaf_Area_index,Land_surface_temperature,Volumetric_Soil_Moisture,Temperature,Precipitation
0,11:06:21,2020-04-06,NaN,NaN,NaN,0.320214,0.436073,0.457228,3.104510,NaN,NaN,69.8,0.0
1,11:06:19,2020-05-01,NaN,NaN,NaN,0.321962,0.435299,0.459722,3.100036,NaN,NaN,77.0,0.0
2,11:06:21,2020-05-06,NaN,NaN,NaN,0.337963,0.442432,0.482572,3.141542,NaN,NaN,80.6,0.0
3,11:06:19,2020-05-11,NaN,NaN,NaN,0.222053,0.385900,0.317063,2.828257,NaN,NaN,71.6,0.0
4,11:06:19,2020-05-21,NaN,NaN,NaN,0.189151,0.368886,0.270085,2.740386,NaN,NaN,NaN,0.0
5,11:06:31,2020-05-26,NaN,NaN,NaN,0.168154,0.353358,0.240104,2.662481,NaN,NaN,89.6,0.0
6,11:06:19,2020-05-31,NaN,NaN,NaN,0.173758,0.358141,0.248105,2.686255,NaN,NaN,66.2,0.0
7,11:06:19,2020-06-10,NaN,NaN,NaN,0.227841,0.376623,0.325330,2.780010,NaN,NaN,73.4,0.0
8,11:06:19,2020-06-20,NaN,NaN,NaN,0.293234,0.417183,0.418704,2.997298,NaN,NaN,77.0,0.0
9,11:06:31,2020-06-25,NaN,NaN,NaN,0.210105,0.339913,0.300006,2.596686,NaN,NaN,75.2,0.0


In [ ]:
new_data = new_data.drop(["Air_temperature", "Precipitation_dataset"], axis = 1)

In [ ]:
new_data.head()

,Hourtime,Date,Soil_temperature,NDVI,NDWI,SAVI,Leaf_Area_index,Land_surface_temperature,Volumetric_Soil_Moisture,Temperature,Precipitation
0,11:06:21,2020-04-06,NaN,0.320214,0.436073,0.457228,3.104510,NaN,NaN,69.8,0.0
1,11:06:19,2020-05-01,NaN,0.321962,0.435299,0.459722,3.100036,NaN,NaN,77.0,0.0
2,11:06:21,2020-05-06,NaN,0.337963,0.442432,0.482572,3.141542,NaN,NaN,80.6,0.0
3,11:06:19,2020-05-11,NaN,0.222053,0.385900,0.317063,2.828257,NaN,NaN,71.6,0.0
4,11:06:19,2020-05-21,NaN,0.189151,0.368886,0.270085,2.740386,NaN,NaN,NaN,0.0
